<a href="https://colab.research.google.com/github/Rano62/projet_python/blob/master/mef_analyse_achats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd 

from google.colab import files
data_to_load = files.upload()
df = pd.read_excel("/content/ligne d'acquisition recherche PV all NAT inf.xlsx")


Saving ligne d'acquisition recherche PV all NAT inf.xlsx to ligne d'acquisition recherche PV all NAT inf.xlsx


In [3]:
data_to_load = files.upload()
df2 = pd.read_excel("/content/Inventaire Vspé(dats).xlsx")

Saving Inventaire Vspé(dats).xlsx to Inventaire Vspé(dats).xlsx


In [8]:
df["Date d'acquisition"] = pd.to_datetime(df["Date d'acquisition"])

df.rename(columns={'Plus/Moins Value Latente - dev réf': 'Plus/Moins Value Latente', 
                           'Valo Clean Détenu - dev val': 'VR', 
                           'Valeur d\'acquisition/Prime bilantaire - dev réf': 'VB', 
                           'Cours de la valeur (place de cotation)': 'Cours', 
                           'Identifiant de l\'actif': 'Code titre'}, inplace=True)

df2.rename(columns  = {'Unnamed: 20': 'Prix'}, inplace = True)
df3=df2[['Code titre','Prix','MCP','Nominal']]
data=pd.merge(df, df3,how="left", on=["Code titre"])

data.loc[(data['NAT2'] == 'OBLIGATION TAUX VARIABLE') |(data['NAT2'] == 'OBLIGATION TAUX FIXE')|(data['NAT2'] == 'PRODUIT STRUCTURE'), 'Prix'] = data['Prix']/100


data.loc[(data['MCP']== 'Piece') & ((data['NAT2'] != 'OBLIGATION TAUX VARIABLE')& (data['NAT2'] != 'OBLIGATION TAUX FIXE')
& (data['NAT2'] != 'PRODUIT STRUCTURE')), 'VRcal'] = data['Prix']*data['Quantité détenue']
#data.loc[(data['MCP']== 'Piece') & (data['NAT2'] != 'OBLIGATION TAUX FIXE'), 'VRcal'] = data['Prix']*data['Quantité détenue']
data.loc[(data['MCP']== 'Piece') & ((data['NAT2'] == 'OBLIGATION TAUX VARIABLE')| (data['NAT2'] == 'OBLIGATION TAUX FIXE')
| (data['NAT2'] == 'PRODUIT STRUCTURE')), 'VRcal'] = data['Prix']*data['Quantité détenue']*data['Nominal']


data.loc[(data['MCP']== 'Nominal') & ((data['NAT2'] == 'OBLIGATION TAUX VARIABLE')| (data['NAT2'] == 'OBLIGATION TAUX FIXE')|
                                      (data['NAT2'] == 'PRODUIT STRUCTURE')), 'VRcal'] = data['Prix']*data['Quantité détenue']

data.loc[(data['Code titre']== 'FR0011198787')|(data['Code titre']== 'FR0011427848')|(data['Code titre']== 'FR0011008705'), 'VRcal'] = data['Prix']*data['Quantité détenue']*data['OATis : indice courant']/data['OATis : indice de base']*data['Nominal']                            

data['PMVL']=data['VRcal']-data['VB']


In [ ]:
data.head(3)

In [ ]:


# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter("pandas_column_formats.xlsx", engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
df.to_excel(writer, sheet_name='Sheet1')

# Get the xlsxwriter workbook and worksheet objects.
workbook  = writer.book
worksheet = writer.sheets['Sheet1']

# Add some cell formats.
format1 = workbook.add_format({'num_format': '#,##0.00'})
format2 = workbook.add_format({'num_format': '0%'})

# Note: It isn't possible to format any cells that already have a format such
# as the index or headers or any cells that contain dates or datetimes.

# Set the column width and format.
worksheet.set_column(1, 1, 18, format1)

# Set the format but not the column width.
worksheet.set_column(2, 2, None, format2)

# Close the Pandas Excel writer and output the Excel file.
writer.save()

In [9]:
data.to_excel('pmvl achats.xlsx', sheet_name = 'Synthèse',index=False)